In [20]:
import relevancer as rlv
import pandas as pd
from sklearn.naive_bayes import MultinomialNB 
from bson.objectid import ObjectId

In [21]:
my_token_pattern=r"[#@]?\w+\b|[\U00010000-\U0010ffff]"

In [22]:
collection = 'coll123' 

In [23]:
rlvdb, rlvcl = rlv.connect_mongodb(configfile='myconfig.ini',coll_name=collection)

In [24]:
begin = ObjectId('55950fb4d04475ee9867f3a4')
end = ObjectId('55950fc9d04475ee986841c3')

annotated_tw_ids = ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960'] #You should get the actual annotated tweet ids from the annotated tweets collection.

tweetlist = rlv.read_json_tweet_fields_database(rlvcl, mongo_query=({'_id': {'$gte': begin, '$lte': end},'lang':'en'}), tweet_count=10000, annotated_ids=annotated_tw_ids)

rlv.logging.info("Number of tweets:" + str(len(tweetlist)))

In [28]:
tweetsDF = rlv.create_dataframe(tweetlist)

9992


In [27]:
tok = rlv.tok_results(tweetsDF, elimrt = True)


Available attributes of the tweets: Index(['id_str', 'is_retweet', 'text', 'texttok', 'is_notrt'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9991
Data columns (total 5 columns):
id_str        9992 non-null object
is_retweet    9992 non-null bool
text          9992 non-null object
texttok       9992 non-null object
is_notrt      9992 non-null bool
dtypes: bool(2), object(3)
memory usage: 331.8+ KB

tweet set summary: None
0    Solar Powered Motion Sensor Security Wall Floo...
1    Brian Williams claimed he saw a 'body floating...
2    High Lumens CREE 36W LED Work Light Bars Flood...
3    Centre to unveil schemes for people affected b...
4    18W CREE 4"x4" LED WORK LIGHT FLOOD BEAM OFFRO...
Name: text, dtype: object

tweets are NOT tokenized.


In [9]:
start_tweet_size = len(tweetsDF)
rlv.logging.info("\nNumber of the tweets after retweet elimination:"+ str(start_tweet_size))

In [10]:
tw_id_list = rlv.get_ids_from_tw_collection(rlvcl)
print ("Length of the tweet ids and the first then ids",len(tw_id_list),tw_id_list[:10])

In get_tw_ids
Length of the tweet ids and the first then ids 20000 ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960']


In [11]:
cluster_list = rlv.create_clusters(tweetsDF, my_token_pattern, nameprefix='1-')

print (len(cluster_list))  
a_cluster = cluster_list[0]

In create_clusters
20


In [14]:
#print("cluster_no", a_cluster['cno'] )

#print("cluster_str", a_cluster['cstr'] )

#print("cluster_tweet_ids", a_cluster['twids'] )

#print("cluster_freq", a_cluster['rif'] )

#print("cluster_prefix", a_cluster['cnoprefix'] )

#print("cluster_tuple_list", a_cluster['ctweettuplelist'] )

In [15]:
collection_name = collection + '_clusters'
rlvdb[collection_name].insert(cluster_list) 
print("Clusters were written to the collection:", collection_name)

Clusters were written to the collection: coll123_clusters


In [16]:
tweets_as_text_label_df = pd.DataFrame({'label' : ['relif', 'social'] , 'text' : ["RT @OliverMathenge: Meanwhile, Kenya has donated Sh91 million to Malawi flood victims, according to the Ministry of Foreign Affairs." , "Yow ehyowgiddii! Hahaha thanks sa flood! #instalike http://t.co/mLaTESfunR"]})
print("tweets_as_text_label_df:", tweets_as_text_label_df)

tweets_as_text_label_df:     label                                               text
0   relif  RT @OliverMathenge: Meanwhile, Kenya has donat...
1  social  Yow ehyowgiddii! Hahaha thanks sa flood! #inst...


In [17]:
# get vectorizer and classifier
vect_and_classifier = rlv.get_vectorizer_and_mnb_classifier(tweets_as_text_label_df, my_token_pattern, pickle_file="vectorizer_and_classifier_dict")
vectorizer, mnb_classifier = vect_and_classifier["vectorizer"], vect_and_classifier["classifier"]

In get_mnb_classifier:
Pickle file was written to vectorizer_and_classifier_dict.pickle


In [18]:
# get label for a new tweet:
ntw = vectorizer.transform(["Why do you guys keep flooding TL with smear campaign for a candidate you dont like.You think you can actually influnece people's decision?"])
predictions = mnb_classifier.predict(ntw)
print("Predictions:", predictions)

Predictions: ['relif']


In [19]:
rlv.logging.info('\nscript finished')